# Gradio Day!

Today we will build User Interfaces using the outrageously simple Gradio framework.

Prepare for joy!

Please note: your Gradio screens may appear in 'dark mode' or 'light mode' depending on your computer settings.

In [4]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
#from openai import OpenAI
#import google.generativeai
import google.generativeai as genai
import anthropic

In [5]:
import gradio as gr # oh yeah!

In [6]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
#openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

#if openai_api_key:
#    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
#else:
#    print("OpenAI API Key not set")
#    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCQ


In [7]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

#openai = OpenAI()

claude = anthropic.Anthropic()

genai.configure()

In [8]:
# A generic system message - no more snarky adversarial AIs!

system_message = "You are a helpful assistant"

In [14]:
# Let's wrap a call to GPT-4o-mini in a simple function

def message_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
    )
    return completion.choices[0].message.content

# Let's wrap a call to GEMINI in a simple function

def message_gemini(prompt):
    model_instance = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message
    )
    # 2 Prepare the request
    user_message = [
        {
            "role": "user",
            "parts": [
                {
                    "text": prompt
                }
            ]
        }
    ]
    #3 call to gemini api
    response = model_instance.generate_content(
        contents=user_message,
        stream=True,
    )
    full_response_text = ""
    # Iterate over the response stream and print each chunk
    for chunk in response:
        if chunk.text:  # Ensure the chunk has text content
            print(chunk.text, end='')  # Print the chunk without a newline
            full_response_text += chunk.text
            
    return full_response_text

# Let's wrap a call to Claude in a simple function

def message_Claude(prompt):
    message = claude.messages.create(
    model="claude-3-7-sonnet-latest",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": prompt},
    ],
    )

    print(message.content[0].text)
    return message.content[0].text


In [15]:
# This can reveal the "training cut off", or the most recent date in the training data

#message_gpt("What is today's date?")
message_gemini("What is today's date?")

Today is October 26, 2023.


'Today is October 26, 2023.\n'

## User Interface time!

In [19]:
# here's a simple function

def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [20]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [20]:
# The simplicty of gradio. This might appear in "light mode" - I'll show you how to make this in dark mode later.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox").launch()

* Running on local URL:  http://127.0.0.1:7866
* To create a public link, set `share=True` in `launch()`.


In [21]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7867
* Running on public URL: https://bec3594d9363a9fe48.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

## Forcing dark mode

Gradio appears in light mode or dark mode depending on the settings of the browser and computer. There is a way to force gradio to appear in dark mode, but Gradio recommends against this as it should be a user preference (particularly for accessibility reasons). But if you wish to force dark mode for your screens, below is how to do it.

In [18]:
# Define this variable and then pass js=force_dark_mode when creating the Interface

force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never", js=force_dark_mode).launch()

NameError: name 'shout' is not defined

In [24]:
# Inputs and Outputs

view = gr.Interface(
    fn=shout,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hi


In [ ]:
# And now - changing the function from "shout" to "message_gpt"

view = gr.Interface(
    #fn=message_gpt,
    #fn=message_Claude,
    fn=message_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Textbox(label="Response:", lines=8)],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


1. Why don't scientists trust atoms? 
Because they make up everything!


2. What do you call a lazy kangaroo? 
 Pouch potato!


3. Why did the bicycle fall over? 
Because it was two tired.


In [22]:
# Let's use Markdown
# Are you wondering why it makes any difference to set system_message when it's not referred to in the code below it?
# I'm taking advantage of system_message being a global variable, used back in the message_gpt function (go take a look)
# Not a great software engineering practice, but quite common during Jupyter Lab R&D!

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    #fn=message_gpt,
    fn=message_gemini,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never",
    js=force_dark_mode
)
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


### Weather Jokes:

* **Why was the meteorologist sad?** Because he had a lot of *low* pressure.

* **What do you call a lazy kangaroo?** Pouch potato.  (Okay, it's a bit of a stretch, but it's related to the weather being *blah* sometimes!)

* **Why did the snowman refuse to go to the party?**  He was afraid of melting into a puddle!

* **What kind of hair do they have on a sunny beach?**  Wavy!

* **What did the cloud say to the other cloud?**  Nothing, it just rained on me!

* **Why did the scarecrow win an award?** Because he was outstanding in his field! (Again, a little agriculture-weather themed).

* **Why don't scientists trust atoms?** Because they make up everything! (A bit of a science/weather connection, as weather involves atoms)


Enjoy!


In [26]:
# Let's create a call that streams back results
# If you'd like a refresher on Generators (the "yield" keyword),
# Please take a look at the Intermediate Python notebook in week1 folder.

def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [ ]:
view = gr.Interface(
    fn=stream_gpt,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [25]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": prompt},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [35]:
def stream_Gemini(prompt):

    model_instance = genai.GenerativeModel(
        model_name="gemini-1.5-flash",
        system_instruction=system_message
    )
    # 2 Prepare the request
    user_message = [
        {
            "role": "user",
            "parts": [
                {
                    "text": prompt
                }
            ]
        }
    ]
    #3 call to gemini api
    result = model_instance.generate_content(
        contents=user_message,
        stream=True,
    )
    full_response_text = ""
    for chunk in result:
        if chunk.text:
            print(chunk.text, end='') 
            full_response_text += chunk.text
            yield full_response_text


In [27]:
view = gr.Interface(
    fn=stream_claude,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [36]:
view = gr.Interface(
    fn=stream_Gemini,
    inputs=[gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


Why don't scientists trust atoms? 

Because they make up everything!


## Minor improvement

I've made a small improvement to this code.

Previously, it had these lines:

```
for chunk in result:
  yield chunk
```

There's actually a more elegant way to achieve this (which Python people might call more 'Pythonic'):

`yield from result`

I cover this in more detail in the Intermediate Python notebook in the week1 folder - take a look if you'd like more.

In [ ]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_Gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [37]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7869
* To create a public link, set `share=True` in `launch()`.


Why don't scientists trust atoms? 

Because they make up everything!
Improving your sleep involves a multifaceted approach targeting your sleep hygiene, environment, and overall health. Here's a breakdown of strategies you can implement:

**1. Optimize Your Sleep Hygiene:**

* **Consistent Sleep Schedule:** Go to bed and wake up around the same time every day, even on weekends, to regulate your body's natural sleep-wake cycle (circadian rhythm).
* **Create a Relaxing Bedtime Routine:** Wind down an hour or two before bed. This could involve a warm bath, reading a book, listening to calming music, or practicing gentle stretching or yoga. Avoid screens (phones, tablets, computers, TV) during this time. The blue light emitted from these devices interferes with melatonin production, a hormone that regulates sleep.
* **Make Your Bedroom Sleep-Conducive:** Ensure your bedroom is dark, quiet, and cool. Consider using blackout curtains, earplugs, or a white noise machine if needed.  A comforta

# Building a company brochure generator

Now you know how - it's simple!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you read the next few cells</h2>
            <span style="color:#900;">
                Try to do this yourself - go back to the company brochure in week1, day5 and add a Gradio UI to the end. Then come and look at the solution.
            </span>
        </td>
    </tr>
</table>

In [38]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [39]:
# With massive thanks to Bill G. who noticed that a prior version of this had a bug! Now fixed.

system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [40]:
def stream_brochure(company_name, url, model):
    prompt = f"Please generate a company brochure for {company_name}. Here is their landing page:\n"
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Claude":
        result = stream_claude(prompt)
    elif model=="Gemini":
        result = stream_Gemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [ ]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["GPT", "Claude", "Gemini"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.


# Hugging Face: The AI Community Platform

**A Brochure for Prospective Customers, Investors, and Recruits**


**For Prospective Customers:**

Hugging Face is the leading platform for collaboration in machine learning. We empower you to:

* **Discover and utilize:** Access over 1 million pre-trained models and 250,000 datasets covering text, image, video, audio, and 3D modalities. Find the perfect tools to jumpstart your AI projects.
* **Collaborate seamlessly:**  Share your work, contribute to the community, and leverage the collective knowledge of a vast network of ML experts.
* **Deploy effortlessly:** Our optimized inference endpoints and Spaces application allow for easy deployment of your models, regardless of your technical expertise.  GPU compute is available starting at $0.60/hour.
* **Accelerate your ML journey:** Leverage our open-source tools and enterprise-grade solutions to build AI faster and more efficiently.


**For Investors:**

Hugging Face is revolutionizing the AI 